In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from vastai.api import VastClient
from io import StringIO

In [4]:
vast = VastClient().authenticate()

api_key_file:  ~/.vast_api_key
Initializing vast.ai client with api_key from /home/sam/.vast_api_key.
Already logged in.


## Search for Available Machines

In [15]:
def filter_offers():
    return [offer for offer in vast.search_offers('dph', 'dph<.12 inet_down>200', instance_type='on-demand') 
        if offer['gpu_name'] and offer['id'] not in []]

In [ ]:
for offer in filter_offers():
    print(offer['id'], offer['dph_total'], offer['gpu_name'], offer['inet_down'], offer['inet_down'])

## Start new Instance with Mongodb

In [7]:
offer = 399889
# offer = filter_offers()[0]['id']
onstart = """#!/bin/sh
touch ~/.no_auto_tmux
echo onstart.sh Starting `date` >> /root/startup.log
apt-get install -y mongodb-server git vim
service mongodb start
pip install --upgrade pip pipenv
"""
resp=vast.create_instance(offer, disk=20, onstart_cmd=onstart, python_utf8=True, lang_utf8=True,
                          image="tensorflow/tensorflow:1.14.0-gpu-py3-jupyter")
mongo_instance_id = resp['new_contract']

mongo_instance = vast.get_instance(mongo_instance_id, retries=4).wait_until_running()

print(mongo_instance.pb_remote['tail']('/var/log/mongodb/mongodb.log','-n2'))

https://vast.ai/api/v0/asks/399889/?api_key=e794a3273e5a3e74d545bbc368849ff263f726042cf92587df9519a6cda62e74 
 {"client_id": "me", "image": "tensorflow/tensorflow:1.14.0-gpu-py3-jupyter", "price": null, "disk": 20, "label": null, "onstart": "#!/bin/sh\ntouch ~/.no_auto_tmux\necho onstart.sh Starting `date` >> /root/startup.log\napt-get install -y mongodb-server git vim\nservice mongodb start\npip install --upgrade pip pipenv\n", "runtype": "ssh", "python_utf8": false, "lang_utf8": false, "use_jupyter_lab": false, "jupyter_dir": null, "create_from": null, "force": false}
{"success": true, "new_contract": 401072}
Waiting 60s...
Waiting 60s...
Waiting 60s...


CommandNotFound: ('tail -n 2', [<RemotePath /usr/local/sbin>, <RemotePath /usr/local/bin>, <RemotePath /usr/sbin>, <RemotePath /usr/bin>, <RemotePath /sbin>, <RemotePath /bin>, <RemotePath /usr/games>, <RemotePath /usr/local/games>])

In [8]:
print(mongo_instance.pb_remote['tail']('/var/log/mongodb/mongodb.log','-n2'))

2019-10-26T16:52:45.730+0000 I INDEX    [thread1] build index done.  scanned 0 total records. 0 secs
2019-10-26T16:52:45.730+0000 I COMMAND  [thread1] command config.$cmd command: createIndexes { createIndexes: "system.sessions", indexes: [ { key: { lastUse: 1 }, name: "lsidTTLIndex", expireAfterSeconds: 1800 } ], $db: "config" } numYields:0 reslen:98 locks:{ Global: { acquireCount: { r: 1, w: 1 } }, Database: { acquireCount: { W: 1 } }, Collection: { acquireCount: { w: 1 } } } protocol:op_msg 136ms



In [9]:
print(mongo_instance.ssh_connection_command)

'ssh root@ssh4.vast.ai -p 11072 -i /home/sam/.ssh/vastai'

In [ ]:
# onstart = "#!/bin/sh\n"\
#           "touch ~/.no_auto_tmux\n"\
#           "apt-get install -y mongodb-server\n"\
#           "mkdir -p ~/.ssh\n"\
#           "ssh-keygen  -P '' -f ~/.ssh/mongo\n"
# resp=vast.create_instance(filter_offers()[0]['id'], disk=1, onstart_cmd=onstart,
#                           image="tensorflow/tensorflow:1.14.0-gpu-py3-jupyter")
# worker_instance_id = resp['new_contract']

# time.sleep(30)
# worker_instance = vast.get_instance(worker_instance_id)

## Install model and download data

In [5]:
# mongo_instance=vast.get_instance(400808)

In [10]:
install_script_target='/root/install_script.sh'
install_script = StringIO("""#!/bin/sh
INSTALL_LOG=/root/install.log
echo avian_vocalizations INSTALL BEGIN `date` >> $INSTALL_LOG 2>&1
cd ~
[ ! -d avian_vocalizations ] && git clone https://github.com/samhiatt/avian_vocalizations.git  >> $INSTALL_LOG 2>&1
cd avian_vocalizations
export LC_ALL=C.UTF-8
export LANG=C.UTF-8
pipenv install >> $INSTALL_LOG 2>&1
pipenv run python setup.py develop >> $INSTALL_LOG 2>&1
pipenv run download_data >> $INSTALL_LOG 2>&1
echo avian_vocalizations INSTALL COMPLETE `date` >> INSTALL_LOG 2>&1
""")
# monitor_output(mongo_instance, install_script)
remote=mongo_instance.pb_remote
remote.sftp.putfo(install_script,install_script_target)
remote['chmod']('+x',install_script_target)
p=remote.session().popen(install_script_target)
p

In [36]:
import time
from IPython.display import clear_output

while True:
    clear_output(wait=True)
    print(remote['tail']('/root/install.log','-n5'))
    ret = p.proc.poll()
    if ret is not None:
        break
    time.sleep(1)
print("Install process has exited with code:",p.proc.poll())

Extracted /root/avian_vocalizations/data/melspectrograms.zip.
Extracted /root/avian_vocalizations/data/feature_shapes.csv.
Extracted contents of xeno-canto-ca-nv.zip to data/audio.
Extracted contents of melspectrograms.zip to data/.
Extracted contents of mfccs.zip to data/.

Install process has exited with code: 0


In [68]:
hyperopt_worker_log="/root/hyperopt_worker.log"
start_hyperopt_worker_target='/root/avian_vocalizations/start_hyperopt_worker.sh'
start_hyperopt_worker="#!/bin/sh\ncd /root/avian_vocalizations\n"\
    "export LC_ALL=C.UTF-8\n"\
    "export LANG=C.UTF-8\n"\
    "nohup pipenv run hyperopt-mongo-worker --mongo=localhost:27017/avian_vocalizations "\
    " --max-consecutive-failures=1 --reserve-timeout=999999 --workdir=."\
    " >> %s 2>&1\n"%hyperopt_worker_log

print(start_hyperopt_worker)
remote.sftp.putfo(StringIO(start_hyperopt_worker),start_hyperopt_worker_target)
remote['chmod']('+x',start_hyperopt_worker_target)

#!/bin/sh
cd /root/avian_vocalizations
export LC_ALL=C.UTF-8
export LANG=C.UTF-8
nohup pipenv run hyperopt-mongo-worker --mongo=localhost:27017/avian_vocalizations  --max-consecutive-failures=1 --reserve-timeout=999999 --workdir=. >> /root/hyperopt_worker.log 2>&1



''

In [69]:
pworker=remote.session().popen(start_hyperopt_worker_target)

In [70]:
while True:
    clear_output(wait=True)
    print(remote['tail'](hyperopt_worker_log,'-n5'))
    ret = pworker.proc.poll()
    if ret is not None:
        break
    time.sleep(1)
print("Hyperopt worker process ended with code",ret)

INFO:hyperopt.mongoexp:no job found, sleeping for 1.3s
INFO:hyperopt.mongoexp:no job found, sleeping for 3.9s
INFO:hyperopt.mongoexp:no job found, sleeping for 2.3s
INFO:hyperopt.mongoexp:no job found, sleeping for 4.6s
INFO:hyperopt.mongoexp:no job found, sleeping for 1.1s

Hyperopt worker process ended with code 0


In [152]:
mongo_instance.ssh_connection_command

'ssh root@ssh4.vast.ai -p 10808 -i /home/sam/.ssh/vastai'

In [10]:
mongo_instance.wait_until_running()

400500: running   $0.0921/hr ssh5.vast.ai:10500  343.6↑ 434.2↓  flops:9.295104T
	1X 7.9GB GTX 1070 Ti, 10.666656/32X 23.8GB AMD Ryzen Threadripper 1950X 16-Core Processor
	Successfully loaded tensorflow/tensorflow:1.14.0-gpu-py3-jupyter

In [26]:
print(mongo_instance.pb_remote['service'](['mongodb','status']))

[' * Checking status of database mongodb', '   ...done.', '']

In [27]:
worker_machine.path('~/.ssh/mongo.pub').isfile()

True

In [28]:
pub_key=worker_machine['cat'](worker_machine.path('~/.ssh/mongo.pub'))
pub_key

'ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQCwQNWg4ud+cP01AhsHEPohGD7/XRafkPiR0PpUrxMX6NOZUgOYuQzK9EN/667HhTnYJm6Yhez/gNGM8/GGzvuDoQQZazEwpnDtCQsgzS9wjBqmxT7pb2jeLuzZGGoXGMN2tY2dNw6TOJvc80QJKtU3hj4NEynTPi96uz888iClf6Gc8/H9rZYkZGF8jBgm5i8ONlbMWDHGlFryOtbuW2XZQZ+Li7ZP8Ad3ozk1pcZIW26IFcihqlxo8sEh+nHyY/Xsyt1bfjy6B7V3rDOBradwxnH5JGiSIZjobLRXrVJ4xxEk/GfJrhywdK3g8lTwVBUdasXJpI08m3P27sszlleB root@b9df8fc9c11d\n'

In [ ]:
mongo_machine.session().run('echo "\n%s" >> %s'%(pub_key, mongo_machine.path('~/.ssh/authorized_keys')))

In [30]:
mongo_machine['cat'](mongo_machine.path('~/.ssh/authorized_keys'))

'ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDaKOIegXeknsbwKsyvY+sdgRFn71kg9yr64ktpV17ImXrggQ+LHmhqMhbv/1gNQhNKIRJz7FllaxxQZkeOnqcv/tLpom3NaZG7OMWkfRR7IK1Apv4n9J2sGUIHlBTyjdDmawiXlttzDjphcR0ZSZ58V+8Hr/S7L+pb1nkXmSY109LV6XjYjo1DPmngU+lnAFilErV3fSZFnmKDA95xQH+ZVN91nKO/jsUOLAIeIo4o5aSR/nv8KGB9GpKbVAueXPji/HrWijXuWSepEtbhtBq8cXTFqgoakhD/Cz4kmZIqwVOLRU+OZRq+xFsJK1UQBR34GRcQeda5AKQ7tDypK2Rj sam@localhost\nssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQCwQNWg4ud+cP01AhsHEPohGD7/XRafkPiR0PpUrxMX6NOZUgOYuQzK9EN/667HhTnYJm6Yhez/gNGM8/GGzvuDoQQZazEwpnDtCQsgzS9wjBqmxT7pb2jeLuzZGGoXGMN2tY2dNw6TOJvc80QJKtU3hj4NEynTPi96uz888iClf6Gc8/H9rZYkZGF8jBgm5i8ONlbMWDHGlFryOtbuW2XZQZ+Li7ZP8Ad3ozk1pcZIW26IFcihqlxo8sEh+nHyY/Xsyt1bfjy6B7V3rDOBradwxnH5JGiSIZjobLRXrVJ4xxEk/GfJrhywdK3g8lTwVBUdasXJpI08m3P27sszlleB root@b9df8fc9c11d\n\n'

In [8]:
mongo_instance.start().wait_until_running(check_every_s=15)
mongo_machine = mongo_instance.pb_remote()
mongo_machine

Starting instance 398753.


<ParamikoMachine paramiko://root@ssh4.vast.ai>

In [ ]:
worker_instance.start().wait_until_running()
worker_machine = mongo_instance.pb_remote()
worker_machine

In [34]:
mongo_machine['cat']("/root/.ssh/authorized_keys")

'ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDaKOIegXeknsbwKsyvY+sdgRFn71kg9yr64ktpV17ImXrggQ+LHmhqMhbv/1gNQhNKIRJz7FllaxxQZkeOnqcv/tLpom3NaZG7OMWkfRR7IK1Apv4n9J2sGUIHlBTyjdDmawiXlttzDjphcR0ZSZ58V+8Hr/S7L+pb1nkXmSY109LV6XjYjo1DPmngU+lnAFilErV3fSZFnmKDA95xQH+ZVN91nKO/jsUOLAIeIo4o5aSR/nv8KGB9GpKbVAueXPji/HrWijXuWSepEtbhtBq8cXTFqgoakhD/Cz4kmZIqwVOLRU+OZRq+xFsJK1UQBR34GRcQeda5AKQ7tDypK2Rj sam@localhost'

In [41]:
list(worker_machine.pgrep(''))

[ProcInfo(1, 0, 'Ss+', '/bin/bash -c while [ ! -e /.launch ]; do sleep 1; done; bash /.launch'),
 ProcInfo(6, 0, 'S+', 'bash /.launch'),
 ProcInfo(16, 0, 'Ss', '/usr/sbin/sshd'),
 ProcInfo(21, 0, 'S+', 'ssh -i /etc/forward_key -N -o TCPKeepAlive=yes -o ServerAliveInterval=10 -o ServerAliveCountMax=2 ubuntu@ssh4.vast.ai -o StrictHostKeyChecking=no -o ConnectTimeout=10 -o RequestTTY=no -o ExitOnForwardFailure=yes -R \\*:38753:localhost:22'),
 ProcInfo(44, 103, 'Sl', '/usr/bin/mongod --config /etc/mongodb.conf'),
 ProcInfo(177, 0, 'Ss', 'sshd: root@notty'),
 ProcInfo(192, 0, 'Ss', '-bash'),
 ProcInfo(198, 0, 'Ss', '/usr/lib/openssh/sftp-server'),
 ProcInfo(200, 0, 'Ss', 'sshd: root@notty'),
 ProcInfo(215, 0, 'Ss', '-bash'),
 ProcInfo(221, 0, 'Ss', '/usr/lib/openssh/sftp-server'),
 ProcInfo(542, 0, 'Rs', '/bin/ps -e -o pid,uid,stat,args')]

In [42]:
vast.stop_all_instances()

Stopping instance 398753.
Stopping instance 398754.


In [150]:
vast.get_instances()

[398753: offline   $0.0252/hr ssh4.vast.ai:38753  423.8↑ 495.8↓  flops:8.736T
 	1X 11.2GB Tesla K80, 16.0/32X 15.7GB Xeon® Silver 4108 
 	Successfully loaded tensorflow/tensorflow:1.14.0-gpu-py3-jupyter,
 398754: offline   $0.0252/hr ssh5.vast.ai:38754  423.8↑ 495.8↓  flops:8.736T
 	1X 11.2GB Tesla K80, 16.0/32X 15.7GB Xeon® Silver 4108 
 	Successfully loaded tensorflow/tensorflow:1.14.0-gpu-py3-jupyter]

In [86]:
inst=vast.get_instances()[0]

In [147]:
inst

398753: offline   $0.0252/hr ssh4.vast.ai:38753  423.8↑ 495.8↓  flops:8.736T
	1X 11.2GB Tesla K80, 16.0/32X 15.7GB Xeon® Silver 4108 
	Successfully loaded tensorflow/tensorflow:1.14.0-gpu-py3-jupyter

In [149]:
vast.stop_all_instances()

Stopping instance 398753.
Stopping instance 398754.
